In [1]:
! wget http://hackzurich.siemens.cool/downloads/rssi.zip
!unzip "/content/rssi.zip" -d "/content/rssi/"


--2021-09-25 22:35:53--  http://hackzurich.siemens.cool/downloads/rssi.zip
Resolving hackzurich.siemens.cool (hackzurich.siemens.cool)... 52.219.74.94
Connecting to hackzurich.siemens.cool (hackzurich.siemens.cool)|52.219.74.94|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752326164 (717M) [application/zip]
Saving to: ‘rssi.zip’

rssi.zip            100%[===================>] 717.47M  29.1MB/s    in 40s     

2021-09-25 22:36:33 (18.1 MB/s) - ‘rssi.zip’ saved [752326164/752326164]

Archive:  /content/rssi.zip
  inflating: /content/rssi/rssi.csv  


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [4]:
rssi_df=pd.read_csv('/content/rssi/rssi.csv')
rssi_df.drop(['ID','AreaNumber','Track', 'Position','PositionNoLeap','Latitude','Longitude','A1_TotalTel','A1_ValidTel','A2_TotalTel','A2_ValidTel'],inplace=True,axis=1)
close_data = rssi_df['A2_RSSI'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]



In [5]:
from keras.models import load_model


In [6]:
model_rssi=load_model('/content/rssi_forcast.h5')
model_events=load_model('/content/model_3_multi.h5')

In [16]:
close_data = close_data.reshape((-1))


def annotate_event(out_predict):
  if out_predict==0:
    return 'No distruption'
  elif out_predict==1:
    return 'Hardwarefehler Bediengeraet'
  elif out_predict==2:
    return 'Hardwarefehler Verteiler'
  elif out_predict==3:
    return 'Position unverifiziert (Eichung falsch)'
  elif out_predict==4:
    return 'Keine Linienleitertelegramme empfangen'
  elif out_predict==5:
    return 'Zwangsbremse wurde aktiviert'
  elif out_predict==6:
    return 'Position unbekannt (ZSI0)'
  elif out_predict==7:
    return 'Stoerung: Zwangsbremse wurde aktiviert'


def predict(num_prediction, model_rssi,model_events):
    look_back=15
    prediction_list = close_data[-look_back:]
    disrupution_events=[]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        x_2=prediction_list[-10:]
        x_2 = x_2.reshape((1, 10, 1))
        out = model_rssi.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
        out = np.argmax(model_events.predict(x_2),axis=1)
        disrupution_events.append(annotate_event(out))
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list,disrupution_events
    
def predict_dates(num_prediction):
    last_date = rssi_df['DateTime'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast_rssi,disrupution_forcasted = predict(num_prediction, model_rssi,model_events)
forecast_dates = predict_dates(num_prediction)

In [17]:
import plotly.graph_objs as go

In [18]:
trace1 = go.Scatter(
    x = forecast_dates,
    y = forecast_rssi,
    mode = 'lines',
    name = 'Data'
)
layout = go.Layout(
    title = "sensor values",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "rssi"}
)
fig = go.Figure(data=[trace1], layout=layout)
fig.show()

In [19]:
disrupution_forcasted

['No distruption',
 'No distruption',
 'No distruption',
 'No distruption',
 'No distruption',
 'No distruption',
 'No distruption',
 'No distruption',
 'No distruption',
 'Zwangsbremse wurde aktiviert',
 'Keine Linienleitertelegramme empfangen',
 'Keine Linienleitertelegramme empfangen',
 'Keine Linienleitertelegramme empfangen',
 'Keine Linienleitertelegramme empfangen',
 'Keine Linienleitertelegramme empfangen',
 'Keine Linienleitertelegramme empfangen',
 'No distruption',
 'No distruption',
 'No distruption',
 'No distruption',
 'No distruption',
 'No distruption',
 'No distruption',
 'No distruption',
 'No distruption',
 'No distruption',
 'No distruption',
 'No distruption',
 'No distruption',
 'No distruption']